In [1]:
#pip install h2o

In [2]:
import h2o

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,27 mins 16 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.4
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_ameyv_rdeami
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.839 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [4]:
df = h2o.import_file('C:/Users/User/Desktop/cdac advance modules/pml lecture/class  practice/program_dataset/Cancer.csv')
df.summary()

Parse progress: |█████████████████████████████████████████████████████████| 100%


,subjid,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
type,int,enum,enum,enum,enum,enum,int,enum,enum,enum,enum
mins,1.0,,,,,,1.0,,,,
mean,143.5,,,,,,2.0489510489510487,,,,
maxs,286.0,,,,,,3.0,,,,
sigma,82.70530011230639,,,,,,0.7382166403717153,,,,
zeros,0,,,,,,0,,,,
missing,0,0,0,0,0,0,0,0,0,0,0
0,1.0,40-49,premeno,15-19,0-2,yes,3.0,right,left_up,no,recurrence-events
1,2.0,50-59,ge40,15-19,0-2,no,1.0,right,central,no,no-recurrence-events
2,3.0,50-59,ge40,35-39,0-2,no,2.0,left,left_low,no,recurrence-events


In [5]:
y_col = 'Class'
x_cols = df.col_names[2:10]

In [6]:
print("Response = " + y_col)
print("Pridictors = " + str(x_cols))

Response = Class
Pridictors = ['menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat']


In [7]:
df['Class'] = df['Class'].asfactor()
df['Class'].levels()

[['no-recurrence-events', 'recurrence-events']]

In [8]:
train,  test = df.split_frame(ratios=[.8])
print(df.shape)
print(train.shape)
#print(valid.shape)
print(test.shape)

(286, 11)
(228, 11)
(58, 11)


In [9]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
glm_logistic = H2OGeneralizedLinearEstimator(family = "binomial")
glm_logistic.train(x=x_cols, y=y_col, training_frame=train, 
                   validation_frame=test, model_id="glm_logistic")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [10]:
y_pred = glm_logistic.predict(test_data=test)

y_pred_df = y_pred.as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%


In [11]:
print(glm_logistic.auc() )
print(glm_logistic.confusion_matrix() )

0.7536764705882353

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3614913885186768: 


,,no-recurrence-events,recurrence-events,Error,Rate
0,no-recurrence-events,137.0,23.0,0.1437,(23.0/160.0)
1,recurrence-events,27.0,41.0,0.3971,(27.0/68.0)
2,Total,164.0,64.0,0.2193,(50.0/228.0)


In [12]:
# Naive Bayes

In [13]:
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator
model_nb = H2ONaiveBayesEstimator()
model_nb.train(x=x_cols, y=y_col, training_frame=train, 
                   validation_frame=test, model_id="model_nb")

naivebayes Model Build progress: |████████████████████████████████████████| 100%


In [14]:
y_prednb = model_nb.predict(test_data=test)

naivebayes prediction progress: |█████████████████████████████████████████| 100%


In [15]:
y_prednb = y_prednb.as_data_frame()

In [16]:
print(model_nb.auc() )
print(model_nb.confusion_matrix() )

0.7782628676470588

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4685150490722178: 


,,no-recurrence-events,recurrence-events,Error,Rate
0,no-recurrence-events,135.0,25.0,0.1562,(25.0/160.0)
1,recurrence-events,27.0,41.0,0.3971,(27.0/68.0)
2,Total,162.0,66.0,0.2281,(52.0/228.0)


In [20]:
from h2o.grid.grid_search import H2OGridSearch
hyper_params = {'mtries':[3,4,5,6]}

In [21]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
model_rf = H2ORandomForestEstimator()
gs = H2OGridSearch(model=model_rf,hyper_params=hyper_params)
gs.train(x=x_cols, y=y_col, training_frame=train, 
                   validation_frame=test)

drf Grid Build progress: |████████████████████████████████████████████████| 100%


In [23]:
y_predrf = gs.predict(test_data=test)
y_predrf = y_predrf.as_data_frame()

drf prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


AttributeError: 'dict' object has no attribute 'as_data_frame'

In [24]:
print(gs.auc() )
print(gs.confusion_matrix() )

{'Grid_DRF_py_5_sid_ad02_model_python_1627724537410_312_model_2': 0.6822610294117647, 'Grid_DRF_py_5_sid_ad02_model_python_1627724537410_312_model_1': 0.6747242647058823, 'Grid_DRF_py_5_sid_ad02_model_python_1627724537410_312_model_3': 0.6957261029411764, 'Grid_DRF_py_5_sid_ad02_model_python_1627724537410_312_model_4': 0.6714613970588236}

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5972222222222222: 


,,no-recurrence-events,recurrence-events,Error,Rate
0,no-recurrence-events,145.0,15.0,0.0938,(15.0/160.0)
1,recurrence-events,39.0,29.0,0.5735,(39.0/68.0)
2,Total,184.0,44.0,0.2368,(54.0/228.0)



Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.21296299828423393: 


,,no-recurrence-events,recurrence-events,Error,Rate
0,no-recurrence-events,91.0,69.0,0.4313,(69.0/160.0)
1,recurrence-events,21.0,47.0,0.3088,(21.0/68.0)
2,Total,112.0,116.0,0.3947,(90.0/228.0)



Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1884057418159817: 


,,no-recurrence-events,recurrence-events,Error,Rate
0,no-recurrence-events,85.0,75.0,0.4688,(75.0/160.0)
1,recurrence-events,16.0,52.0,0.2353,(16.0/68.0)
2,Total,101.0,127.0,0.3991,(91.0/228.0)



Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.15000000000000002: 


,,no-recurrence-events,recurrence-events,Error,Rate
0,no-recurrence-events,74.0,86.0,0.5375,(86.0/160.0)
1,recurrence-events,15.0,53.0,0.2206,(15.0/68.0)
2,Total,89.0,139.0,0.443,(101.0/228.0)


{'Grid_DRF_py_5_sid_ad02_model_python_1627724537410_312_model_2': , 'Grid_DRF_py_5_sid_ad02_model_python_1627724537410_312_model_1': , 'Grid_DRF_py_5_sid_ad02_model_python_1627724537410_312_model_3': , 'Grid_DRF_py_5_sid_ad02_model_python_1627724537410_312_model_4': }
